In [1]:
# the aim is to end up with train, test, evaluate
import sys
sys.path.append("/Users/david/Documents/code/fhmm/v1")

In [2]:
import src.ads_davidjames9610.ads as ads
import src.ads_davidjames9610.useful as useful
import numpy as np
import os
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
from importlib import reload
import importlib
import src.classifiers_davidjames9610.test_a.config as base_config
reload(base_config)
config = importlib.import_module(base_config.config_location)
reload(config)
print(config.basedir)
import src.misc_davidjames9610.fe_methods as fe
import src.misc_davidjames9610.proc_methods as pm

/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/whale_data


"""
we want to end up with this

    section 2, samples to features,
    -- choose fe_method,
    -- noise and reverb, apply to test / evaluate before fe method,

    select fe_methods,
    -- this point also is important for choosing cross-validation, and train-test-eval split
    - fe_methods = ['mfcc', 'log-power', 'lfcc']
    - select noise or reverb to be applied to eval and test
    - post_proc = ['noise', 'reverb']

    -- labels don't change

    features = {
        'mfcc':
            'train': [],
            'train_labels': [cv_index_0] [cv_index_1]
            'test': [],
            'test_labels': []   // test and eval will need different process method if noise is being added
            'eval': [],
            'eval_labels': []
        'lfcc':
            ...
    }
"""

In [3]:
import pickle
sls = pickle.load(open(config.samples_labels,'rb'))

In [ ]:
# going to have to do fe after cross-validation split
import src.misc_davidjames9610.cross_validation as cv
from src.misc_davidjames9610.cross_validation import *
import importlib
importlib.reload(cv)
import b_samples_to_features
reload(b_samples_to_features)
import src.misc_davidjames9610.noisey as noisey
reload(noisey)
import random

# define config things
n_folds = 2
cv_output = cv.split_data_for_cross_validation(sls['samples'], sls['labels'], n_folds) # , test_ratio=0.8) # could update test ratio to
signal_average_power = b_samples_to_features.get_average_power_for_samples(cv_output)
signal_average_power_db = 10 * np.log10(signal_average_power)
max_sample_length = np.max([len(sample) for sample in sls['samples']])


In [ ]:
from src.misc_davidjames9610.noisey import get_noise_avg_watts
import importlib
importlib.reload(fe)
nfft = 128

class ProcessingBase:
    def __init__(self, fe_method):
        self.fe_method = fe_method
    def __str__(self):
        return self.fe_method.__str__()
    def pre_process(self, sample):
        return self.fe_method(sample)
    def post_process(self, sample):
        return self.fe_method(sample)
    def get_noise_feature(self):
        pass

class ProcessingGaussNoise:
    def __init__(self, fe_method, snr, signal_power, sample_len):
        self.fe_method = fe_method
        self.snr = snr
        self.signal_power = signal_power
        self.noise_sample = noisey.get_noise_for_sample(sample_len, self.signal_power,self.snr, self.snr, self.snr)

    def __str__(self):
        return self.fe_method.__str__() + '_GaussNoise_SNR' + str(self.snr)
    def pre_process(self, sample):
        return self.fe_method(sample)
    def post_process(self, sample):
        sample = self.noise_sample[:len(sample)] + sample
        return self.fe_method(sample)
    def get_noise_feature(self):
        return self.fe_method(self.noise_sample)

class ProcessingRealNoise:
    def __init__(self, fe_method, snr, signal_power, sample_len, noise_key):

        self.fe_method = fe_method
        self.snr = snr
        self.signal_power = signal_power
        self.noise_key = noise_key

        self.noise_sample = b_samples_to_features.get_real_noise_sample(
            noise_key=base_config.noise_sound_lib[noise_key], target_snr_db=snr, signal_db=signal_average_power, sample_len=sample_len, sr=config.sr)

    def __str__(self):
        return self.fe_method.__str__() + '_RealNoise_' + self.noise_key + '_SNR' + str(self.snr)

    def pre_process(self, sample):
        return self.fe_method(sample)
    def post_process(self, sample):
        sample = self.noise_sample[:len(sample)] + sample
        return self.fe_method(sample)
    def get_noise_feature(self):
        return self.fe_method(self.noise_sample)

class ProcessingGaussNoiseReverb:
    # todo finish this somehow
    def __init__(self, fe_method, snr, signal_power, sample_len):
        self.snr = snr
        self.signal_power = signal_power
        self.fe_method = fe_method
    def __str__(self):
        return self.fe_method.__str__() + '_GaussNoise_SNR' + str(self.snr) + '_Reverb'
    def pre_process(self, sample):
        return self.fe_method(sample)
    def post_process(self, sample):
        # todo update so Gauss Noise Reverb works
        return self.fe_method(sample)
    def get_noise_feature(self):
        # update
        return []


In [ ]:
proc_methods = [
    # ProcessingBase(
    #     fe_method=fe.FeatureExtractorLogPower(nfft=128)),
    ProcessingBase(
        fe_method=fe.FeatureExtractorLogPower(nfft=128)),
    ProcessingGaussNoise(
        fe_method=fe.FeatureExtractorLogPower(nfft=128),
        snr=40,
        signal_power=signal_average_power,
        sample_len=max_sample_length),
    ProcessingGaussNoise(
        fe_method=fe.FeatureExtractorLogPower(nfft=128),
        snr=20,
        signal_power=signal_average_power,
        sample_len=max_sample_length),
    ProcessingRealNoise(
        fe_method=fe.FeatureExtractorLogPower(nfft=128),
        snr=40,
        signal_power=signal_average_power,
        sample_len=max_sample_length,
        noise_key='UNDER_WATER_DRONE'
        ),
    ProcessingRealNoise(
        fe_method=fe.FeatureExtractorLogPower(nfft=128),
        snr=20,
        signal_power=signal_average_power,
        sample_len=max_sample_length,
        noise_key='UNDER_WATER_DRONE'
        )
    # ProcessingGaussNoise(
    #     fe_method=fe.FeatureExtractorLogPower(nfft=128),
    #     snr=20,
    #     signal_power=signal_average_power,
    #     sample_len=max_sample_length),
    # ProcessingGaussNoise(
    #     fe_method=fe.FeatureExtractorLogPower(nfft=128),
    #     snr=10,
    #     signal_power=signal_average_power,
    #     sample_len=max_sample_length),

    # ProcessingBase(
    #     fe_method=fe.FeatureExtractorMfcc(nfft=128, num_ceps=8, fs=config.sr)),
    # ProcessingGaussNoise(
    #     fe_method=fe.FeatureExtractorMfcc(nfft=128, num_ceps=8, fs=config.sr),
    #     snr=60,
    #     signal_power=signal_average_power,
    #     sample_len=max_sample_length),
    #     ProcessingGaussNoise(
    #     fe_method=fe.FeatureExtractorMfcc(nfft=128, num_ceps=8, fs=config.sr),
    #     snr=40,
    #     signal_power=signal_average_power,
    #     sample_len=max_sample_length),
    #     ProcessingGaussNoise(
    #     fe_method=fe.FeatureExtractorMfcc(nfft=128, num_ceps=8, fs=config.sr),
    #     snr=20,
    #     signal_power=signal_average_power,
    #     sample_len=max_sample_length),
    #     ProcessingGaussNoise(
    #     fe_method=fe.FeatureExtractorMfcc(nfft=128, num_ceps=8, fs=config.sr),
    #     snr=10,
    #     signal_power=signal_average_power,
    #     sample_len=max_sample_length),
    ]

for proc_method in proc_methods:
    print(proc_method.__str__())


KeyboardInterrupt: 

In [ ]:
reload(fe)

features = {
}

import b_samples_to_features
reload(b_samples_to_features)

for proc_method in proc_methods:

    fe_name = proc_method.__str__()
    features[fe_name] = {}
    print('processing for: ', fe_name)
    n_folds = len(cv_output['train_data'])

    # train
    train_features = []
    train_labels = []
    for i in range(n_folds):
        curr_features = [proc_method.pre_process(sample) for sample in cv_output['train_data'][i]]
        curr_labels = cv_output['train_label'][i]
        train_features.append(curr_features)
        train_labels.append(curr_labels)
    features[fe_name]['train_features'] = train_features
    features[fe_name]['train_label'] = train_labels

    # val
    val_features = []
    val_labels = []
    val_noise = []
    for i in range(n_folds):
        curr_features = []
        curr_noise_features = []

        for sample in cv_output['val_data'][i]:
            curr_feature = proc_method.post_process(sample)
            curr_features.append(curr_feature)

        curr_noise_features.append(proc_method.get_noise_feature())

        curr_labels = cv_output['val_label'][i]
        val_features.append(curr_features)
        val_labels.append(curr_labels)
        val_noise.append(curr_noise_features)

    features[fe_name]['val_features'] = val_features
    features[fe_name]['val_label'] = cv_output['val_label']
    features[fe_name]['noise_features'] = val_noise


In [ ]:
import pickle
pickle.dump(features, open(config.features,'wb'))

In [ ]:
config.features